In [147]:
from bs4 import BeautifulSoup
from selenium import webdriver

from pymongo import MongoClient
import psycopg2 as pg2
from sqlalchemy import create_engine
from os import environ

from pprint import pprint
import pandas as pd
import time

In [148]:
# from https://towardsdatascience.com/creating-an-easy-website-scraper-for-data-science-sports-prediction-pt-1-f024abd53861
def url_to_soup(url: str) -> BeautifulSoup:
    driver = webdriver.Chrome()
    driver.get(url)
    source = driver.page_source
    driver.close()

    url_soup = BeautifulSoup(source, 'lxml')
    return url_soup

In [149]:
def make_url(year: int, page: int=0) -> str:
    url = f'http://www.nhl.com/stats/teams?aggregate=0&reportType=game&seasonFrom={year}{year+1}&seasonTo={year}{year+1}&dateFromSeason&gameType=2&filter=gamesPlayed,gte,1&sort=points,wins&page={page}&pageSize=100'
    return url

In [150]:
def convert_double_dash(cell: str, type_constructor):
    if cell == '--':
        return None
    return type_constructor(cell)

In [151]:
def make_postgres_conn(dbname='postgres', port=5432):
    conn = pg2.connect(
        dbname=dbname,
        port=port,
        host=os.environ['PG_HOST'],
        user=os.environ['PG_USER'],
        password=os.environ['PG_PASSWORD'])
    return conn

In [152]:
def make_alchemy_engine(dbname='postgres', port=5432):
    username = os.environ['PG_USER']
    password = os.environ['PG_PASSWORD']
    host = os.environ['PG_HOST']
    string = f'postgresql://{username}:{password}@{host}:{port}/{dbname}'
    return create_engine(string)

In [153]:
def extract_page_table(soup: BeautifulSoup, season: int, page: int, row_schema: dict) -> list:
    rows = soup.find_all('div', class_='rt-tr-group')
    all_rows = []
    for row in rows:
        values = row.find_all('div', class_='rt-td')
        if values[0].text.strip() == '&nbsp;' or values[0].text.strip() == '':
            break

        new_row = row_schema.copy()
        new_row['team'] = values[1].text.strip()
        new_row['game'] = values[2].text.strip()
        new_row['gp'] = int(values[3].text.strip())
        new_row['wins'] = int(values[4].text.strip())
        new_row['losses'] = int(values[5].text.strip())
        new_row['ties'] = convert_double_dash(values[6].text.strip(), int)
        new_row['ot_losses'] = int(values[7].text.strip())
        new_row['points'] = int(values[8].text.strip())
        new_row['point_percent'] = float(values[9].text.strip())
        new_row['reg_wins'] = int(values[10].text.strip())
        new_row['reg_ot_wins'] = int(values[11].text.strip())
        new_row['so_wins'] = int(values[12].text.strip())
        new_row['gf'] = int(values[13].text.strip())
        new_row['ga'] = int(values[14].text.strip())
        new_row['gf_per_gp'] = convert_double_dash(values[15].text.strip(), float)
        new_row['ga_per_gp'] = convert_double_dash(values[16].text.strip(), float)
        new_row['pp_percent'] = convert_double_dash(values[17].text.strip(), float)
        new_row['pk_percent'] = convert_double_dash(values[18].text.strip(), float)
        new_row['pp_net_percent'] = convert_double_dash(values[19].text.strip(), float)
        new_row['pk_net_percent'] = convert_double_dash(values[20].text.strip(), float)
        new_row['sf_per_gp'] = convert_double_dash(values[21].text.strip(), float)
        new_row['sa_per_gp'] = convert_double_dash(values[22].text.strip(), float)
        new_row['fo_win_percent'] = float(values[23].text.strip())
        new_row['season'] = season
        new_row['page'] = page

        all_rows.append(new_row)
    
    return all_rows

In [66]:
url = 'http://www.nhl.com/stats/teams?aggregate=0&reportType=game&seasonFrom=20172018&seasonTo=20172018&dateFromSeason&gameType=2&filter=gamesPlayed,gte,1&sort=points,wins&page=0&pageSize=100'

In [ ]:
test = url_to_soup(url)
test.find_all('div', class_='rt-tr-group')[0].find_all('div', class_='rt-td')[1].text.strip() # 'Toronto Maple Leafs'
test.find('span', class_='-totalPages').text.strip() # '26'

In [68]:
row = test.find_all('div', class_='rt-tr-group')[0].find_all('div', class_='rt-td')
for cell in row:
    print(cell.text.strip())

1
Toronto Maple Leafs
2017/11/16vs NJD
1
1
0
--
0
2
1.000
0
1
0
1
0
1.00
0.00
0.0
100.0
0.0
100.0
25.0
42.0
55.1


In [112]:
empty_row = {'team': None, 'game': None, 'gp': None, 'wins': None, 'losses': None, 'ties': None,
            'ot_losses': None, 'points': None, 'point_percent': None, 'reg_wins': None, 
            'reg_ot_wins': None, 'so_wins': None, 'gf': None, 'ga': None, 'gf_per_gp': None, 
            'ga_per_gp': None, 'pp_percent': None, 'pk_percent': None, 'pp_net_percent': None, 
            'pk_net_percent': None, 'sf_per_gp': None, 'sa_per_gp': None, 
            'fo_win_percent': None, 'season': None, 'page': None}

rows = test.find_all('div', class_='rt-tr-group')
all_rows = []
for row in rows:
    values = row.find_all('div', class_='rt-td')
    if values[0].text.strip() == '&nbsp;' or values[0].text.strip() == '':
        break

    new_row = empty_row.copy()
    new_row['team'] = values[1].text.strip()
    new_row['game'] = values[2].text.strip()
    new_row['gp'] = int(values[3].text.strip())
    new_row['wins'] = int(values[4].text.strip())
    new_row['losses'] = int(values[5].text.strip())
    new_row['ties'] = convert_double_dash(values[6].text.strip(), int)
    new_row['ot_losses'] = int(values[7].text.strip())
    new_row['points'] = int(values[8].text.strip())
    new_row['point_percent'] = float(values[9].text.strip())
    new_row['reg_wins'] = int(values[10].text.strip())
    new_row['reg_ot_wins'] = int(values[11].text.strip())
    new_row['so_wins'] = int(values[12].text.strip())
    new_row['gf'] = int(values[13].text.strip())
    new_row['ga'] = int(values[14].text.strip())
    new_row['gf_per_gp'] = convert_double_dash(values[15].text.strip(), float)
    new_row['ga_per_gp'] = convert_double_dash(values[16].text.strip(), float)
    new_row['pp_percent'] = convert_double_dash(values[17].text.strip(), float)
    new_row['pk_percent'] = convert_double_dash(values[18].text.strip(), float)
    new_row['pp_net_percent'] = convert_double_dash(values[19].text.strip(), float)
    new_row['pk_net_percent'] = convert_double_dash(values[20].text.strip(), float)
    new_row['sf_per_gp'] = convert_double_dash(values[21].text.strip(), float)
    new_row['sa_per_gp'] = convert_double_dash(values[22].text.strip(), float)
    new_row['fo_win_percent'] = float(values[23].text.strip())
    new_row['season'] = 2017
    new_row['page'] = 0
    all_rows.append(new_row)

In [113]:
pd.DataFrame(all_rows)

,team,game,gp,wins,losses,ties,ot_losses,points,point_percent,reg_wins,...,ga_per_gp,pp_percent,pk_percent,pp_net_percent,pk_net_percent,sf_per_gp,sa_per_gp,fo_win_percent,season,page
0,Toronto Maple Leafs,2017/11/16vs NJD,1,1,0,None,0,2,1.0,0,...,0.0,0.0,100.0,0.0,100.0,25.0,42.0,55.1,2017,0
1,Columbus Blue Jackets,2017/12/09vs ARI,1,1,0,None,0,2,1.0,1,...,0.0,0.0,100.0,0.0,100.0,34.0,35.0,57.1,2017,0
2,Nashville Predators,2017/10/19@ PHI,1,1,0,None,0,2,1.0,1,...,0.0,0.0,100.0,0.0,100.0,24.0,28.0,45.8,2017,0
3,Chicago Blackhawks,2018/01/30@ NSH,1,1,0,None,0,2,1.0,1,...,1.0,0.0,100.0,0.0,100.0,31.0,43.0,43.1,2017,0
4,Chicago Blackhawks,2017/11/18@ PIT,1,1,0,None,0,2,1.0,1,...,1.0,40.0,100.0,20.0,100.0,38.0,36.0,42.0,2017,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Montréal Canadiens,2018/01/19@ WSH,1,1,0,None,0,2,1.0,1,...,2.0,25.0,66.7,25.0,66.7,26.0,26.0,42.9,2017,0
96,Montréal Canadiens,2017/10/05@ BUF,1,1,0,None,0,2,1.0,0,...,2.0,0.0,80.0,0.0,100.0,40.0,45.0,50.0,2017,0
97,Montréal Canadiens,2018/02/04vs OTT,1,1,0,None,0,2,1.0,1,...,1.0,66.7,100.0,66.7,100.0,35.0,26.0,42.1,2017,0
98,Anaheim Ducks,2018/01/19vs LAK,1,1,0,None,0,2,1.0,1,...,1.0,0.0,100.0,0.0,100.0,31.0,24.0,61.8,2017,0


In [78]:
years = range(2009, 2018+1)
urls = []
for year in years:
    urls.append(make_url(year))
urls

['http://www.nhl.com/stats/teams?aggregate=0&reportType=game&seasonFrom=20092010&seasonTo=20092010&dateFromSeason&gameType=2&filter=gamesPlayed,gte,1&sort=points,wins&page=0&pageSize=100',
 'http://www.nhl.com/stats/teams?aggregate=0&reportType=game&seasonFrom=20102011&seasonTo=20102011&dateFromSeason&gameType=2&filter=gamesPlayed,gte,1&sort=points,wins&page=0&pageSize=100',
 'http://www.nhl.com/stats/teams?aggregate=0&reportType=game&seasonFrom=20112012&seasonTo=20112012&dateFromSeason&gameType=2&filter=gamesPlayed,gte,1&sort=points,wins&page=0&pageSize=100',
 'http://www.nhl.com/stats/teams?aggregate=0&reportType=game&seasonFrom=20122013&seasonTo=20122013&dateFromSeason&gameType=2&filter=gamesPlayed,gte,1&sort=points,wins&page=0&pageSize=100',
 'http://www.nhl.com/stats/teams?aggregate=0&reportType=game&seasonFrom=20132014&seasonTo=20132014&dateFromSeason&gameType=2&filter=gamesPlayed,gte,1&sort=points,wins&page=0&pageSize=100',
 'http://www.nhl.com/stats/teams?aggregate=0&reportType

In [154]:
mongo = MongoClient('localhost', 27017)
db = mongo['nhl']
coll = db['soup']

In [137]:
# conn = make_postgres_conn(dbname='nhl')

In [138]:
# conn.close()

In [155]:
engine = make_alchemy_engine('nhl')
# engine.dispose()

In [156]:
empty_row = {'team': None, 'game': None, 'gp': None, 'wins': None, 'losses': None, 'ties': None,
            'ot_losses': None, 'points': None, 'point_percent': None, 'reg_wins': None, 
            'reg_ot_wins': None, 'so_wins': None, 'gf': None, 'ga': None, 'gf_per_gp': None, 
            'ga_per_gp': None, 'pp_percent': None, 'pk_percent': None, 'pp_net_percent': None, 
            'pk_net_percent': None, 'sf_per_gp': None, 'sa_per_gp': None, 
            'fo_win_percent': None, 'season': None, 'page': None}

str

In [157]:
season = 2009
start_url = make_url(season)
soup = url_to_soup(start_url)
num_pages = int(soup.find('span', class_='-totalPages').text.strip())
for page in range(num_pages):
    if page > 0:
        new_url = make_url(season, page)
        soup = url_to_soup(new_url)
    
    coll.insert_one({'season': season, 'page': page, 'soup': soup.prettify()})

    all_rows = extract_page_table(soup, season, page, empty_row)

    table = pd.DataFrame(all_rows)
    table.to_sql('games', engine, index=False, if_exists='append')
    time.sleep(5)